In [1]:
from pyspark import SparkContext
import json
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

In [2]:
f= open('/yelp/yelp_academic_dataset_review.json')
print json.loads(f.readline())
f.close()

{u'votes': {u'funny': 0, u'useful': 0, u'cool': 0}, u'user_id': u'PUFPaY9KxDAcGqfsorJp3Q', u'review_id': u'Ya85v4eqdd6k9Od8HbQjyA', u'text': u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.', u'business_id': u'5UmKMjUEUNdYWqANhGckJw', u'stars': 4, u'date': u'2012-08-01', u'type': u'review'}


In [3]:
sc = SparkContext()

In [4]:
lines = sc.textFile("/yelp/yelp_academic_dataset_review.json")

In [5]:
reviews = lines.map(lambda x: json.loads(x))

In [6]:
full_reviews = reviews.filter(
    lambda x: x['votes']['funny'] and x['votes']['useful'] and x['votes']['cool'])

In [7]:
parsedData = full_reviews.map(
    lambda x: 
    LabeledPoint(
        float(x['stars']), 
        [
            float(x['votes']['funny']),
            float(x['votes']['useful']),
            float(x['votes']['cool'])
        ]
    )
)

In [11]:
model = LogisticRegressionWithLBFGS.train(parsedData, numClasses=5)

Py4JJavaError: An error occurred while calling o86.trainLogisticRegressionModelWithLBFGS.
: org.apache.spark.SparkException: Input validation failed.
	at org.apache.spark.mllib.regression.GeneralizedLinearAlgorithm.run(GeneralizedLinearAlgorithm.scala:251)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:94)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainLogisticRegressionModelWithLBFGS(PythonMLLibAPI.scala:293)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractComma

In [9]:
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.353928802833
